In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

In [2]:
#path to sqlite
data_base_path=("Resources/hawaii.sqlite")
print(data_base_path)

Resources/hawaii.sqlite


In [3]:
#make engine
engine = create_engine(f"sqlite:///{data_base_path}", pool_pre_ping=True)
conn=engine.connect()

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [4]:
#query all records in database
data=engine.execute('select * from measurement, station')
for record in data:
    print(record)

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [ ]:
#read through pandas
measurement_data=pd.read_sql("select * from measurement", conn)
measurement_data.head()

In [ ]:
station_data=pd.read_sql("select * from station", conn)
station_data.head()

In [ ]:
# reflect an existing database into a new model 
Base= automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to DB
session=Session(bind=engine)

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
months = session.query(Measurement.date).\
        order_by(Measurement.date.desc()).first()
months

In [ ]:
# Calculate the date 1 year ago from the last data point in the database
one_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print("Query Date: ", one_year_ago)

In [ ]:
# Perform a query to retrieve the data and precipitation score
results = session.query(Measurement.date, Measurement.prcp).\
        filter(Measurement.date >= one_year_ago).\
        order_by(Measurement.date).all()

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
results_df = pd.DataFrame(year_query, columns=["Date", "Precipitation"])
results_df.head()

In [ ]:
results_df.set_index("Date", inplace=True)
results_df.head()

In [ ]:
# Sort the dataframe by date
results_df= results_df.sort_values(by='date')

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
fig, ax =plt.subplots(figsize=(16, 10))
df.plot(ax=ax, x_compat=True, color=black)
plt.title("Precipitation 08-23-2016 - 08-23-2017")
plt.xlabel("Date")
plt.xticks(rotation= 45)
plt.ylable("Precipitation (inches)")
plt.tight_layout()
plt.savefig("prcp.png")
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
stats=results_df.describe
stats

In [ ]:
# Design a query to show how many stations are available in this dataset?
avail_stations = session.query(Measurement.station).distinct().count()
print(f"Stations Available: {avail_stations} ")

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
active_stations = session.query(Measurement.station, 
                                func.count(Measurement.station)).group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()
print(f"Most Active Stations")
active_stations

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?
most_active_station=active_stations[0][0]

most_active_temps = session.query(func.min(Measurement.tobs), func.max(Measurement.tobs),
                                  func.avg(Measurement.tobs)).filter(Measurement.station == most_active_station).all()
print(f"Most Active Station Temperatures")
print(f"Low: {most_active_temps[0][0]} High: {most_active_temps[0][1]} Average: {round(most_active_temps[0][2], 1)}")

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
USC00519281_query = session.query(Measurement.tobs).\
        filter(Measurement.date >= one_year_ago).\
        filter(Measurement.station =="USC00519281").\
        order_by(Measurement.date).all()
USC00519281_df = pd.DataFrame(USC00519281_query, columns=["Tobs"])
USC00519281_df.head()

In [ ]:
USC00519281_df.plot.hist(title ="Data for Station USC00519281", bins=12, legend="upper right")
plt.xlabel = ("Temperature") 
plt.ylabel = ("Frequency")
plt.savefig("Images/station_temp_observation.png")
plt.show()